In [1]:
from matplotlib import pyplot as plt 
import numpy as np 
from skimage import morphology
from skimage import segmentation
from joblib import Parallel, delayed
import multiprocessing


In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]) 



In [3]:
def create_mask(im): 
    im_col = plt.imread(im)
    im_gray = rgb2gray(im_col)
    mask = im_gray < 120 
    struct_el = morphology.disk(5)
    opened = morphology.opening(mask,struct_el)
    tmp = segmentation.slic(im_col, n_segments = 500, sigma = 1, compactness = 10, start_label = 1)
    
    # plt.imshow(segmentation.mark_boundaries(opened,tmp), cmap="gray")
    # white_list = []
    # black_list = []
    # for i in tmp:
    #     for j in i: 
            
    return (tmp, opened)


In [4]:
tmp, mask = create_mask("../data/example_image/ISIC_0001769.jpg")


In [5]:
tmp_list = []
tmp_dict = {}
for i in range(len(tmp)):
    for j in range(len(tmp[i])): 
        if tmp[i][j] not in tmp_list:
            tmp_list.append(tmp[i][j])
            tmp_dict[str(tmp[i][j])] = {
                "Coordinates": [(i,j)],
                "Values": [mask[i][j]],
                "Sum": 0,
                "Result": 0
            } 
        else: 
            tmp_dict[str(tmp[i][j])]["Coordinates"].append((i,j))
            tmp_dict[str(tmp[i][j])]["Values"].append(mask[i][j])

for i in tmp_list:
    tmp_dict[str(i)]["Sum"] = sum(tmp_dict[str(i)]["Values"])
    if tmp_dict[str(i)]["Sum"] > len(tmp_dict[str(i)]["Values"])/2:
        tmp_dict[str(i)]["Result"] = 1
            # if mask[i][j]:
            #     tmp_list.append(j) 
        

In [28]:
cluster_dict = {}
for i in tmp_dict.keys(): 
    tmp_dict[i]["Edges"] = []
    if tmp_dict[i]["Result"] == 1: 
        cluster_dict[i] = tmp_dict[i]



def processInput(coordinates, coordinate):
    edge_list = []
    for k in coordinates: 
        if k[0] >= coordinate[0] + 2 or k[0] <= coordinate[0] -2 or k[1] >= coordinate[1] + 2 or k[1] <= coordinate[1] -2:
            continue 
        if k[0] in (coordinate[0]-1,coordinate[0]+1) and k[1] == coordinate[1]: 
            edge_list.append(k)
        elif k[1] in (coordinate[1]-1,coordinate[1]+1) and k[0] == coordinate[0]:
            edge_list.append(k) 
        if len(edge_list) == 4: 
            break 
    if len(edge_list) < 4: 
        return coordinate 
    else: 
        return 
    
num_cores = multiprocessing.cpu_count()
for i in cluster_dict.keys(): 
    print(i)
    edges = Parallel(n_jobs=num_cores)(delayed(processInput)(cluster_dict[i]["Coordinates"], j) for j in cluster_dict[i]["Coordinates"])
    edges = [i for i in edges if i != None]
    # print(edges[0])
    cluster_dict[i]["Edges"] = edges
    print(f"{i} - {len(cluster_dict[i]['Edges'])} - {len(cluster_dict[i]['Coordinates'])}")
    # for j in tmp_dict[i]["Coordinates"]: 






# results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)


185
185 - 600 - 38265
192
192 - 623 - 11723
194
194 - 715 - 10977
199
199 - 736 - 13308
204
204 - 803 - 14446
222
222 - 636 - 16423
230
230 - 554 - 13141
241
241 - 675 - 13890
242
242 - 693 - 12890
245
245 - 529 - 10297
404
404 - 522 - 16131
405
405 - 561 - 13370
406
406 - 525 - 15308
408
408 - 712 - 14670
413
413 - 573 - 13133
423
423 - 773 - 11882
428
428 - 577 - 14369
429
429 - 686 - 15506
430
430 - 973 - 17092
431
431 - 569 - 15651
434
434 - 624 - 14720
436
436 - 532 - 15401
438
438 - 550 - 16138
442
442 - 769 - 15305
446
446 - 807 - 13086
453
453 - 649 - 18508
454
454 - 560 - 14852
456
456 - 582 - 17971
457
457 - 774 - 15485
459
459 - 633 - 16856
460
460 - 620 - 16088
461
461 - 625 - 15107
462
462 - 669 - 14363
467
467 - 484 - 10428
471
471 - 555 - 14615
474
474 - 667 - 15271
477
477 - 591 - 13351


In [29]:
# print(cluster_dict.keys())

dict_keys(['185', '192', '194', '199', '204', '222', '230', '241', '242', '245', '404', '405', '406', '408', '413', '423', '428', '429', '430', '431', '434', '436', '438', '442', '446', '453', '454', '456', '457', '459', '460', '461', '462', '467', '471', '474', '477'])


In [33]:
cluster_list = []
checked = []
for i in cluster_dict.keys(): 
    checked.append(i)
    for j in cluster_dict.keys(): 
        if j in checked: 
            continue 
        # print(f"{i}, {j}")
        clustered = False 
        for k in cluster_dict[i]["Edges"]: 
            for l in cluster_dict[j]["Edges"]:
                if k[0] == l[0] and k[1] in (l[1]-1, l[1]+1):
                    cluster_list.append((i,j))
                    clustered = True 
                    break 
                elif k[1] == l[1] and k[0] in (l[0]-1, l[0]+1):
                    cluster_list.append((i,j))
                    clustered = True 
                    break 
                    
            if clustered: 
                break 
            


185, 192
185, 194
185, 199
185, 204
185, 222
185, 230
185, 241
185, 242
185, 245
185, 404
185, 405
185, 406
185, 408
185, 413
185, 423
185, 428
185, 429
185, 430
185, 431
185, 434
185, 436
185, 438
185, 442
185, 446
185, 453
185, 454
185, 456
185, 457
185, 459
185, 460
185, 461
185, 462
185, 467
185, 471
185, 474
185, 477
192, 194
192, 199
192, 204
192, 222
192, 230
192, 241
192, 242
192, 245
192, 404
192, 405
192, 406
192, 408
192, 413
192, 423
192, 428
192, 429
192, 430
192, 431
192, 434
192, 436
192, 438
192, 442
192, 446
192, 453
192, 454
192, 456
192, 457
192, 459
192, 460
192, 461
192, 462
192, 467
192, 471
192, 474
192, 477
194, 199
194, 204
194, 222
194, 230
194, 241
194, 242
194, 245
194, 404
194, 405
194, 406
194, 408
194, 413
194, 423
194, 428
194, 429
194, 430
194, 431
194, 434
194, 436
194, 438
194, 442
194, 446
194, 453
194, 454
194, 456
194, 457
194, 459
194, 460
194, 461
194, 462
194, 467
194, 471
194, 474
194, 477
199, 204
199, 222
199, 230
199, 241
199, 242
199, 245
1

In [41]:
tmp_list_two = []
for i in cluster_list: 
    if tmp_list_two == []: 
            tmp_list_two.append([i[0],i[1]]) 
            continue 
    added = True 
    for j in tmp_list_two:
        
        if i[0] in j and i[1] in j: 
            added = False
        elif i[0] in j: 
            j.append(i[1])
            added = False 
        elif i[1] in j:
            j.append(i[0])
            added = False
        else: 
            pass 
    if added: 
        tmp_list_two.append([i[0],i[1]])
        

In [44]:
print(tmp_list_two)

[['185', '192', '199', '204', '222', '194', '241', '230', '242', '245'], ['404', '405', '406', '429', '431', '436', '408', '413', '423', '442', '446', '428', '438', '430', '434', '453', '459', '454', '460', '467', '462', '471', '456', '461', '474', '457', '477']]
